## Recurent Neural Networks

<img src = "Introduction to RNN.jpg">
<img src = "Introduction to RNN1.jpg">
<img src = "Introduction to RNN2.jpg">
<img src = "Introduction to RNN3.jpg">
<img src = "Introduction to RNN4.jpg">
<img src = "Introduction to RNN5.jpg">
<img src = "Introduction to RNN6.jpg">
<img src = "Introduction to RNN7.jpg">
<img src = "Introduction to RNN8.jpg">
<img src = "Introduction to RNN9.jpg">
<img src = "Introduction to RNN10.jpg">
<img src = "Introduction to RNN11.jpg">
<img src = "Introduction to RNN12.jpg">
<img src = "Introduction to RNN13.jpg">
<img src = "Introduction to RNN14.jpg">
<img src = "Introduction to RNN15.jpg">
<img src = "Introduction to RNN16.jpg">

# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras import initializers

#import keras
#from keras.preprocessing import sequence
#from keras.models import Sequential
#from keras.layers import Dense, Embedding
#from keras.layers import SimpleRNN
#from keras.datasets import imdb
#from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17473536/17464789 [==============================] - 4s 0us/step
25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [8]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [9]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5)                 280       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [10]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

c:\Users\vijay\anaconda3\envs\data_science\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [13]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 13s 16ms/step - loss: 0.3595 - accuracy: 0.8402 - val_loss: 0.4491 - val_accuracy: 0.7890
Epoch 2/10
782/782 [==============================] - 14s 18ms/step - loss: 0.3542 - accuracy: 0.8433 - val_loss: 0.4493 - val_accuracy: 0.7911
Epoch 3/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3488 - accuracy: 0.8467 - val_loss: 0.4511 - val_accuracy: 0.7915
Epoch 4/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3449 - accuracy: 0.8486 - val_loss: 0.4497 - val_accuracy: 0.7923
Epoch 5/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3411 - accuracy: 0.8507 - val_loss: 0.4512 - val_accuracy: 0.7920
Epoch 6/10
782/782 [==============================] - 14s 18ms/step - loss: 0.3379 - accuracy: 0.8530 - val_loss: 0.4502 - val_accuracy: 0.7929
Epoch 7/10
782/782 [==============================] - 14s 18ms/step - loss: 0.3348 - accuracy: 0.8541 - val_loss: 0.4526 - val_accuracy:

In [14]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 3ms/step - loss: 0.4543 - accuracy: 0.7942: 1s - loss: 0.4639 - accuracy: 
Test score: 0.45426100492477417
Test accuracy: 0.794160008430481


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [24]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [25]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [26]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [27]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 29s 35ms/step - loss: 0.6247 - accuracy: 0.6542 - val_loss: 0.5629 - val_accuracy: 0.7073
Epoch 2/10
782/782 [==============================] - 26s 34ms/step - loss: 0.4727 - accuracy: 0.7784 - val_loss: 0.4662 - val_accuracy: 0.7777
Epoch 3/10
782/782 [==============================] - 26s 34ms/step - loss: 0.4011 - accuracy: 0.8196 - val_loss: 0.4431 - val_accuracy: 0.8012
Epoch 4/10
782/782 [==============================] - 27s 34ms/step - loss: 0.3584 - accuracy: 0.8452 - val_loss: 0.4245 - val_accuracy: 0.8110
Epoch 5/10
782/782 [==============================] - 26s 33ms/step - loss: 0.3263 - accuracy: 0.8615 - val_loss: 0.3878 - val_accuracy: 0.8266
Epoch 6/10
782/782 [==============================] - 26s 33ms/step - loss: 0.3022 - accuracy: 0.8720 - val_loss: 0.3723 - val_accuracy: 0.8346
Epoch 7/10
782/782 [==============================] - 28s 36ms/step - loss: 0.2854 - accuracy: 0.8816 - val_loss: 0.3708 - val_accuracy:

In [28]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [29]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [30]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [31]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 21s 25ms/step - loss: 0.6560 - accuracy: 0.5971 - val_loss: 0.6138 - val_accuracy: 0.7029
Epoch 2/10
782/782 [==============================] - 19s 24ms/step - loss: 0.5897 - accuracy: 0.7503 - val_loss: 0.5830 - val_accuracy: 0.7493
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.5571 - accuracy: 0.7922 - val_loss: 0.5612 - val_accuracy: 0.7946
Epoch 4/10
782/782 [==============================] - 18s 23ms/step - loss: 0.5353 - accuracy: 0.8101 - val_loss: 0.5477 - val_accuracy: 0.7991
Epoch 5/10
782/782 [==============================] - 17s 22ms/step - loss: 0.5166 - accuracy: 0.8218 - val_loss: 0.5338 - val_accuracy: 0.8053
Epoch 6/10
782/782 [==============================] - 20s 25ms/step - loss: 0.5012 - accuracy: 0.8286 - val_loss: 0.5302 - val_accuracy: 0.79449 - accuracy: 
Epoch 7/10
782/782 [==============================] - 20s 26ms/step - loss: 0.4865 - accuracy: 0.8334 - val_loss: 0.5176 -

In [23]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3039 - accuracy: 0.8707 - val_loss: 0.3682 - val_accuracy: 0.8380
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2984 - accuracy: 0.8749 - val_loss: 0.3658 - val_accuracy: 0.8386
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2942 - accuracy: 0.8782 - val_loss: 0.3621 - val_accuracy: 0.8398
Epoch 4/10
782/782 [==============================] - 19s 24ms/step - loss: 0.2903 - accuracy: 0.8777 - val_loss: 0.3611 - val_accuracy: 0.8409
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2866 - accuracy: 0.8802 - val_loss: 0.3588 - val_accuracy: 0.8412
Epoch 6/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2840 - accuracy: 0.8803 - val_loss: 0.3638 - val_accuracy: 0.8395
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2812 - accuracy: 0.8820 - val_loss: 0.3755 - val_accuracy:

---

#### Long-Short Term memory Networks

<img src = "LSTM.jpg">
<img src = "LSTM1.jpg">
<img src = "LSTM2.jpg">
<img src = "LSTM3.jpg">
<img src = "LSTM4.jpg">
<img src = "LSTM5.jpg">
<img src = "LSTM6.jpg">
<img src = "LSTM7.jpg">
<img src = "LSTM8.jpg">
<img src = "LSTM9.jpg">
<img src = "LSTM10.jpg">
<img src = "LSTM11.jpg">

### Gated Recurrent Unit

<img src = "GRU.jpg">
<img src = "GRU1.jpg">
<img src = "GRU2.jpg">

### Sequence2Sequence

<img src = "S2S.jpg">
<img src = "S2S1.jpg">
<img src = "S2S2.jpg">
<img src = "S2S3.jpg">
<img src = "S2S4.jpg">

### Beam Search

<img src = "Beam Search.jpg">

### Attention

<img src = "Attention.jpg">
<img src = "Attention1.jpg">
<img src = "Attention2.jpg">

### Common Business Applications

<img src = "Applications.jpg">